<a href="https://colab.research.google.com/github/GuillermoCarbajal/J-MOPD/blob/main/J-MOPD_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Joint Motion Offset Prediction and Deblurring (J-MOPD) demo

## Clone the repo

In [ ]:
!git clone https://github.com/GuillermoCarbajal/J-MOPD.git
%cd J-MOPD

## Setup the environment

In [ ]:
!pip install -r 'requirements.txt'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.4/899.4 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121:
      Successfully uninstalled torchvision-0.20.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.13.2 requires torch>=1.13.0, but you have torch 1.11.0 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch

## Download pretrained models

In [ ]:
import gdown
gdown.download('http://iie.fing.edu.uy/~carbajal/J-MOPD/crop320/camera_shake_crop320.pkl', "./pretrained_models/", quiet=False)
gdown.download('http://iie.fing.edu.uy/~carbajal/J-MOPD/crop320/camera_shake_crop320_G.pkl', "./pretrained_models/", quiet=False)

In [ ]:
import torch
from skimage.io import imread
from skimage.transform import rescale
from utils.homographies import compute_intrinsics, generate_video, get_offsets_from_positions, \
                               save_kernels_from_offsets, show_kernels_from_offsets_on_blurry_image, save_motion_flow, reblur_offsets
import numpy as np
#from models.network_nimbusr_pmbm import NIMBUSR_PMBM as net
from models.network_nimbusr_offsets import NIMBUSR_Offsets as net_nimbusr_offsets

from utils.visualization import save_image, tensor2im, save_video, sort_positions, show_positions_found
import os
import argparse
from models.CameraShakeModel_OffsetNet import OffsetNet_quad as OffsetsNet
import json
from PIL import Image
import cv2

## Offsets Computation

In [ ]:
#input_image = 'testing_imgs/0316.png'
input_image ='testing_imgs/manmade_01_gyro_01.png'
camera_model_file = 'pretrained_models/camera_shake_epoch50_epoch35_epoch40_epoch70_epoch70_epoch60_epoch4_epoch30_epoch15_epoch30_epoch30_epoch30_epoch30_epoch27_epoch27_epoch27.pkl'
restoration_model_file ='pretrained_models/camera_shake_epoch50_epoch35_epoch40_epoch70_epoch70_epoch60_epoch4_epoch30_epoch15_epoch30_epoch30_epoch30_epoch30_epoch27_epoch27_epoch27_G.pkl'
n_positions= 25 # number of positions estimated by the network
if torch.cuda.is_available():
  offsets_net = OffsetsNet(n_offset=n_positions,offset_mode='raw').cuda()
  state_dict = torch.load(camera_model_file )
else:
  offsets_net = OffsetsNet(n_offset=n_positions,offset_mode='raw').to('cpu')
  state_dict = torch.load(camera_model_file, map_location=torch.device('cpu') )
  print('cuda is not available')


offsets_net.load_state_dict(state_dict)
offsets_net.eval()

cuda is not available


OffsetNet_quad(
  (pad_1): ReflectionPad2d((1, 2, 1, 2))
  (todepth): SpaceToDepth()
  (conv_1): Sequential(
    (0): ReflectionPad2d((2, 2, 2, 2))
    (1): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
    (2): ReLU(inplace=True)
    (3): Sequential(
      (0): ResnetBlock_woNorm(
        (conv_block): Sequential(
          (0): ReplicationPad2d((2, 2, 2, 2))
          (1): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
          (2): ReLU(inplace=True)
          (3): ReplicationPad2d((2, 2, 2, 2))
          (4): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
        )
      )
      (1): ResnetBlock_woNorm(
        (conv_block): Sequential(
          (0): ReplicationPad2d((2, 2, 2, 2))
          (1): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
          (2): ReLU(inplace=True)
          (3): ReplicationPad2d((2, 2, 2, 2))
          (4): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
        )
      )
      (2): ResnetBlock_woNorm(
        (conv_block): Sequential(
 

In [ ]:
def compute_offsets(input_file, offsets_net, output_dir='testing_results',
                       gamma_factor=1.0, save_video=False):

  n_positions= 25 # number of positions estimated by the network
  if not os.path.exists(output_dir):
      os.makedirs(output_dir)

  img_name = input_file.split('/')[-1]
  img_name, ext = img_name.split('.')

  print('loading image ',input_file)
  blurry_image = imread(input_file)/255.0

  # Blurry image is transformed to pytorch format
  if torch.cuda.is_available():
    blurry_tensor = torch.from_numpy(blurry_image).permute(2,0,1)[None].cuda().float()
  else:
    blurry_tensor = torch.from_numpy(blurry_image).permute(2,0,1)[None].float()

  _, C,H,W = blurry_tensor.shape

  # Kernels and masks are estimated
  blurry_tensor_to_compute_kernels = blurry_tensor**gamma_factor - 0.5


  with torch.no_grad():
      blurry_tensor_ph = blurry_tensor**gamma_factor
      offsets = offsets_net(blurry_tensor_ph - 0.5)
      offsets = offsets[:,:,:H,:W]
      blurry_tensor = blurry_tensor[:,:,:H,:W]


  offsets = offsets.reshape(1,2*n_positions, H,W)
  offsets_BT=None


  kernels_file = os.path.join(output_dir, img_name + '_kernels_found.png' )


  show_kernels_from_offsets_on_blurry_image(blurry_tensor[0],offsets[0].reshape(n_positions,2,H,W), kernels_file)
  print('Kernels saved in ',os.path.join(output_dir, img_name + '_kernels.png') )


In [ ]:
compute_offsets(input_image, offsets_net)

loading image  testing_imgs/manmade_01_gyro_01.png
Kernels saved in  testing_results/manmade_01_gyro_01_kernels.png


## Image Deblurring

In [ ]:
#from models.network_nimbusr_pmbm import NIMBUSR_PMBM as net
from models.network_nimbusr_offsets import NIMBUSR_Offsets as net_nimbusr_offsets

In [ ]:
def load_nimbusr_net(restoration_network_file, type='offsets'):
    opt_net = { "n_iter": 8
        , "h_nc": 64
        , "in_nc": 4 #2 if args.gray else 4 #4
        , "out_nc":3 #1 if args.gray else 3 #3
        #, "ksize": 25
        , "nc": [64, 128, 256, 512]
        , "nb": 2
        , "gc": 32
        , "ng": 2
        , "reduction" : 16
        , "act_mode": "R"
        , "upsample_mode": "convtranspose"
        , "downsample_mode": "strideconv"}


    if type=='pmbm':
        netG = net_nimbusr_offsets(n_iter=opt_net['n_iter'],
                    h_nc=opt_net['h_nc'],
                    in_nc=opt_net['in_nc'],
                    out_nc=opt_net['out_nc'],
                    nc=opt_net['nc'],
                    nb=opt_net['nb'],
                    act_mode=opt_net['act_mode'],
                    downsample_mode=opt_net['downsample_mode'],
                    upsample_mode=opt_net['upsample_mode']
                    )
    elif type=='offsets':
        netG = net_nimbusr_offsets(n_iter=opt_net['n_iter'],
            h_nc=opt_net['h_nc'],
            in_nc=opt_net['in_nc'],
            out_nc=opt_net['out_nc'],
            nc=opt_net['nc'],
            nb=opt_net['nb'],
            act_mode=opt_net['act_mode'],
            downsample_mode=opt_net['downsample_mode'],
            upsample_mode=opt_net['upsample_mode']
            )

    if os.path.exists(restoration_network_file):
        print('Loading model for G [{:s}] ...'.format(restoration_network_file))
        if torch.cuda.is_available():
          print('Running on gpu.')
          netG.load_state_dict(torch.load(restoration_network_file))
          netG = netG.to('cuda')
        else:
          print('Running on cpu. Cuda is not available.')
          netG.load_state_dict(torch.load(restoration_network_file,map_location=torch.device('cpu')))

    else:
        print('Model does not exists')



    return netG


In [ ]:
def restore_images(input_file, offsets_net, netG, output_folder='testing_results', gamma_factor=1.0,
                   resize_factor=1, network_type='nimbusr_sat', focal_length=-1, save_video=False):

  n_positions= 25 # number of positions estimated by the network
  if not os.path.exists(output_folder):
      os.makedirs(output_folder)

  img_name = input_file.split('/')[-1]
  img_name, ext = img_name.split('.')

  print('loading image ',input_file)
  blurry_image = imread(input_file)/255.0

  # Blurry image is transformed to pytorch format
  if torch.cuda.is_available():
    blurry_tensor = torch.from_numpy(blurry_image).permute(2,0,1)[None].cuda().float()
  else:
    blurry_tensor = torch.from_numpy(blurry_image).permute(2,0,1)[None].float()

  _, C,H,W = blurry_tensor.shape

  # Kernels and masks are estimated
  blurry_tensor_to_compute_kernels = blurry_tensor**gamma_factor - 0.5


  with torch.no_grad():
      blurry_tensor_ph = blurry_tensor**gamma_factor
      offsets = offsets_net(blurry_tensor_ph - 0.5)
      offsets = offsets[:,:,:H,:W]
      blurry_tensor = blurry_tensor[:,:,:H,:W]


  offsets = offsets.reshape(1,2*n_positions, H,W)
  offsets_BT=None



  noise_level = 0.01
  noise_level = torch.FloatTensor([noise_level]).view(1,1,1).to(blurry_tensor.device)

  with torch.no_grad():
    output_ph = netG(blurry_tensor_ph[:,:,:H,:W], offsets[:,:,:H,:W], sf=1, sigma=noise_level[None,:])
    output = torch.clamp(output_ph,0,1)**(1.0/gamma_factor)

  output_img = tensor2im(torch.clamp(output[0].detach(),0,1) - 0.5)
  save_image(output_img, os.path.join(output_folder, img_name + '_PMBM.png' ))

  save_motion_flow(offsets.reshape(n_positions,2,H,W), os.path.join(output_folder, f'{img_name}_motion_flow.png'))
  #save_kernels_from_offsets(offsets, os.path.join(output_folder, f'{img_name}_kernels.png'))

  show_kernels_from_offsets_on_blurry_image(blurry_tensor[0],offsets[0].reshape(n_positions,2,H,W), os.path.join(output_folder, img_name + '_kernels.png' ))


  save_image((255*blurry_image).astype(np.uint8), os.path.join(output_folder, img_name + '.png' ))
  print('Output saved in ', os.path.join(output_folder, img_name + '_offsets.png' ))


  #kh, kw = int(np.sqrt(n_positions)),int(np.sqrt(n_positions))   # we have 25 offsets
  #weight = torch.ones((C,1,kh, kw)).cuda()/(kh*kw)
  #weight = torch.zeros((C,1,kh, kw)).cuda()
  #weight[:,0,4,4]=1
  #mask = torch.ones(1, kh * kw, H, W).cuda()

  #reblurred_image_ph = deform_conv2d(output_ph, offsets_dc[:,:,:H,:W], weight, padding=kh//2)
  reblurred_image_ph = reblur_offsets(output_ph, offsets.reshape(1,n_positions,2,H,W))
  reblurred_image = torch.clamp(reblurred_image_ph,0,1)**(1.0/gamma_factor)
  reblurred = tensor2im(torch.clamp(reblurred_image[0,:,:H,:W].detach(),0,1) - 0.5)
  save_image(reblurred, os.path.join(output_folder, img_name + '_reblurred.png' ))


In [ ]:
input_image = 'testing_imgs/manmade_01_gyro_01.png'


In [ ]:
netG = load_nimbusr_net(restoration_model_file, 'offsets')
netG.eval()
restore_images(input_image, offsets_net, netG)

Loading model for G [pretrained_models/camera_shake_epoch50_epoch35_epoch40_epoch70_epoch70_epoch60_epoch4_epoch30_epoch15_epoch30_epoch30_epoch30_epoch30_epoch27_epoch27_epoch27_G.pkl] ...
Running on cpu. Cuda is not available.
loading image  testing_imgs/manmade_01_gyro_01.png
Output saved in  testing_results/manmade_01_gyro_01_offsets.png


## Try with your own images

In [ ]:
from google.colab import files
import shutil

upload_folder = 'upload/input'
result_folder = 'upload/output'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(upload_folder)
os.makedirs(result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

In [ ]:
import glob
input_list = sorted(glob.glob(os.path.join(upload_folder, '*')))
for input_path in input_list:
  img_input = imread(input_path)
  restore_images(input_image, offsets_net, netG)